In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cobra
from cobra.io import read_sbml_model, write_sbml_model
from cobra.flux_analysis import flux_variability_analysis
from tqdm import tqdm

from imatpy.model_utils import read_model
from imatpy.parse_gpr import gene_to_rxn_weights
from imatpy.imat import imat

In [39]:
Table = pd.read_csv("../data/iMat_modified/WT_vs_Ecol_ratio1_3.csv")
Table

,Unnamed: 0,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Gene_name,GeneID,orgdb_symbol,orgdb_old_MXAN,orgdb_Name,iMat
0,1,WP_002614080.1,17681.559540,1.131486,0.333820,3.389505,0.000700,0.004017,MXAN_RS25275,41362483.0,MXAN_RS25275,MXAN_5201,30S ribosomal protein S21,0
1,2,WP_002614803.1,13055.865845,-0.279785,0.341135,-0.820161,0.412124,0.556348,MXAN_RS16100,41360674.0,infA,MXAN_3321,translation initiation factor IF-1,0
2,3,WP_002633201.1,520.781351,-0.488659,0.310281,-1.574894,0.115281,0.223685,MXAN_RS31645,41363738.0,MXAN_RS31645,MXAN_6531,HPr family phosphocarrier protein,0
3,4,WP_002633598.1,17667.143024,1.786742,0.432319,4.132931,0.000036,0.000318,MXAN_RS16040,41360662.0,MXAN_RS16040,MXAN_3309,50S ribosomal protein L14,1
4,5,WP_002633601.1,4426.400261,0.810463,0.545744,1.485060,0.137528,0.253668,MXAN_RS16025,41360659.0,MXAN_RS16025,MXAN_3306,50S ribosomal protein L16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6886,6887,WP_141276995.1,55.570762,2.380438,0.546006,4.359732,0.000013,0.000133,NaN,NaN,NaN,NaN,NaN,-1
6887,6888,WP_141277062.1,25.241491,1.382630,0.762769,1.812645,0.069887,0.155014,NaN,NaN,NaN,NaN,NaN,-1
6888,6889,WP_143049088.1,2729.231930,0.100695,0.293071,0.343586,0.731158,0.819387,MXAN_RS19445,41361331.0,NaN,NaN,NaN,0
6889,6890,WP_143049089.1,109.891161,2.017890,0.643239,3.137076,0.001706,0.008435,NaN,NaN,NaN,NaN,NaN,-1


In [ ]:
iMatTable = pd.Series()
for i in range(len(Table.index)):
    iMatTable = iMatTable[Table['orgdb_old_MXAN'][i]] = Table['iMat'][i] # to fix

iMatTable

TypeError: 'numpy.int64' object does not support item assignment

In [ ]:
# Read in the model
M_xanthus = read_model("../M_xanthus_model.sbml")

# Create a pandas Series representing gene expression weights
model_weights = pd.Series({
           "g_A_imp": 1,
           "g_B_imp": -1,
           "g_C_imp": -1,
           "g_F_exp": 0,
           "g_G_exp": -1,
           "g_H_exp": 0,
           "g_A_B_D_E": 0,
           "g_C_E_F": -1,
           "g_C_H": 0,
           "g_D_G": 1,
       })

# Convert the gene weights into reaction weights
reaction_weights = gene_to_rxn_weights(model=M_xanthus, gene_weights=model_weights)

# Print the reaction weights
print(reaction_weights)

g_A_imp      1
g_B_imp     -1
g_C_imp     -1
g_F_exp      0
g_G_exp     -1
g_H_exp      0
g_A_B_D_E    0
g_C_E_F     -1
g_C_H        0
g_D_G        1
dtype: int64


In [ ]:
# Read in the reaction weights
rxn_weights = pd.read_csv("./tests/data/test_model_reaction_weights.csv", index_col=0).squeeze()

# Run iMAT
imat_results = imat(model=M_xanthus, rxn_weights=rxn_weights, epsilon=1, threshold=0.01)

# Print the imat objective
print(f"iMAT Objective: {imat_results.objective_value}")

# Print the imat flux distribution
print(f"iMAT Flux Distribution: \n{imat_results.fluxes}")